🔹 Objective (Report Text You Can Use)

The objective of this task is to study and implement multiple transformer-based methods to identify differences between two
texts. Instead of using OpenAI APIs, Hugging Face transformer models are used to compute semantic similarity, lexical
difference, factual difference, and logical contradiction. The system is evaluated on sentences and documents such as resumes.

In [ ]:
!pip install -U sentence-transformers transformers pdfplumber scikit-learn torch numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 113.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1

In [ ]:
import re
import numpy as np
import torch
import torch.nn.functional as F
import pdfplumber

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModelForSequenceClassification

3️⃣ Load Transformer Models

Explanation (Report Text)

Sentence-BERT is used to generate sentence embeddings for semantic comparison.
BART-MNLI is used for Natural Language Inference to detect contradictions.

In [ ]:
# Semantic embedding model
semantic_model = SentenceTransformer("all-MiniLM-L6-v2")

# NLI model for contradiction detection
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-mnli")
nli_model = AutoModelForSequenceClassification.from_pretrained("facebook/bart-large-mnli")
nli_model.eval()


BartForSequenceClassification(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
       

4️⃣ Semantic Difference (Embedding-Based)
Explanation

Semantic difference is computed using cosine similarity between sentence embeddings. A threshold converts similarity scores into a binary decision.

In [ ]:
def semantic_difference(text_a, text_b, threshold=0.8):
    emb_a = semantic_model.encode(text_a)
    emb_b = semantic_model.encode(text_b)

    similarity = cosine_similarity([emb_a], [emb_b])[0][0]

    return {
        "similarity_score": round(float(similarity), 4),
        "semantic_difference": similarity < threshold
    }


5️⃣ Lexical (Exact Text) Difference
Explanation

Lexical difference measures changes in wording without understanding meaning.

In [ ]:
import difflib

def lexical_difference(text_a, text_b):
    ratio = difflib.SequenceMatcher(None, text_a, text_b).ratio()
    return {
        "lexical_similarity": round(ratio, 4),
        "lexical_difference": ratio < 0.9
    }


6️⃣ Factual Difference (Numbers & Quantities)
Explanation

Factual differences are detected by extracting numerical values from text.

In [ ]:
def extract_numbers(text):
    return re.findall(r"\d+\.?\d*", text)

def factual_difference(text_a, text_b):
    nums_a = extract_numbers(text_a)
    nums_b = extract_numbers(text_b)

    return {
        "numbers_text_a": nums_a,
        "numbers_text_b": nums_b,
        "factual_difference": nums_a != nums_b
    }


7️⃣ Contradiction Detection (NLI – Stable Version)
Explanation

Natural Language Inference classifies the relationship between two texts as entailment, neutral, or contradiction.

In [ ]:
def contradiction_difference(text_a, text_b):
    inputs = tokenizer(
        text_a,
        text_b,
        return_tensors="pt",
        truncation=True
    )

    with torch.no_grad():
        logits = nli_model(**inputs).logits

    probs = F.softmax(logits, dim=1)[0]
    labels = ["contradiction", "neutral", "entailment"]

    scores = dict(zip(labels, probs.tolist()))

    return {
        "relationship": max(scores, key=scores.get),
        "confidence": round(max(scores.values()), 4),
        "contradiction": scores["contradiction"] > 0.5
    }


8️⃣ Combine All Differences (CORE SYSTEM)
Explanation

Multiple difference signals are combined to obtain a robust comparison result.

In [ ]:
def full_text_difference(text_a, text_b):
    return {
        "semantic": semantic_difference(text_a, text_b),
        "lexical": lexical_difference(text_a, text_b),
        "factual": factual_difference(text_a, text_b),
        "contradiction": contradiction_difference(text_a, text_b)
    }


9️⃣ Test on Sentences (Baseline Experiment)

In [ ]:
text_a = "The model achieved 92% accuracy and passed all tests."
text_b = "The system achieved 90% accuracy and failed several tests."

full_text_difference(text_a, text_b)


{'semantic': {'similarity_score': 0.7434, 'semantic_difference': np.True_},
 'lexical': {'lexical_similarity': 0.7928, 'lexical_difference': True},
 'factual': {'numbers_text_a': ['92'],
  'numbers_text_b': ['90'],
  'factual_difference': True},
 'contradiction': {'relationship': 'contradiction',
  'confidence': 0.9992,
  'contradiction': True}}

🔟 Threshold Sensitivity Analysis
Explanation

Threshold analysis studies how similarity decisions change with different thresholds.

In [ ]:
def threshold_analysis(text_a, text_b, thresholds=[0.6, 0.7, 0.8, 0.9]):
    results = {}
    for t in thresholds:
        results[t] = semantic_difference(text_a, text_b, threshold=t)
    return results


In [ ]:
threshold_analysis(text_a, text_b)

{0.6: {'similarity_score': 0.7434, 'semantic_difference': np.False_},
 0.7: {'similarity_score': 0.7434, 'semantic_difference': np.False_},
 0.8: {'similarity_score': 0.7434, 'semantic_difference': np.True_},
 0.9: {'similarity_score': 0.7434, 'semantic_difference': np.True_}}

In [ ]:
def threshold_analysis_full(text_a, text_b, thresholds=[0.6, 0.7, 0.8, 0.9]):
    results = {}
    for t in thresholds:
        results[t] = {
            "semantic": semantic_difference(text_a, text_b, threshold=t),
            "lexical": lexical_difference(text_a, text_b),
            "factual": factual_difference(text_a, text_b),
            "contradiction": contradiction_difference(text_a, text_b)
        }
    return results


In [ ]:
result = threshold_analysis_full(text_a, text_b)
print(result)


{0.6: {'semantic': {'similarity_score': 0.7434, 'semantic_difference': np.False_}, 'lexical': {'lexical_similarity': 0.7928, 'lexical_difference': True}, 'factual': {'numbers_text_a': ['92'], 'numbers_text_b': ['90'], 'factual_difference': True}, 'contradiction': {'relationship': 'contradiction', 'confidence': 0.9992, 'contradiction': True}}, 0.7: {'semantic': {'similarity_score': 0.7434, 'semantic_difference': np.False_}, 'lexical': {'lexical_similarity': 0.7928, 'lexical_difference': True}, 'factual': {'numbers_text_a': ['92'], 'numbers_text_b': ['90'], 'factual_difference': True}, 'contradiction': {'relationship': 'contradiction', 'confidence': 0.9992, 'contradiction': True}}, 0.8: {'semantic': {'similarity_score': 0.7434, 'semantic_difference': np.True_}, 'lexical': {'lexical_similarity': 0.7928, 'lexical_difference': True}, 'factual': {'numbers_text_a': ['92'], 'numbers_text_b': ['90'], 'factual_difference': True}, 'contradiction': {'relationship': 'contradiction', 'confidence': 0

1️⃣1️⃣ PDF Resume Comparison (Document-Level)
Explanation

PDF documents are converted to plain text before applying NLP models.

In [ ]:

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            if page.extract_text():
                text += page.extract_text() + "\n"
    return text

def clean_text(text):
    text = text.replace("\n", " ")
    return " ".join(text.split())


Example Usage

In [ ]:
resume_old = clean_text(extract_text_from_pdf("VaishnavMankar (1).pdf"))
resume_new = clean_text(extract_text_from_pdf("VaishnavMankar (2).pdf"))

full_text_difference(resume_old, resume_new)


{'semantic': {'similarity_score': 0.995, 'semantic_difference': np.False_},
 'lexical': {'lexical_similarity': 0.6533, 'lexical_difference': True},
 'factual': {'numbers_text_a': ['919146790431',
   '2002',
   '2024',
   '2026',
   '6.55',
   '2020',
   '2024',
   '6.34',
   '2025',
   '2025',
   '2025',
   '2025',
   '8',
   '2024',
   '2024'],
  'numbers_text_b': ['919146790431',
   '2002',
   '2024',
   '2026',
   '6.55',
   '2020',
   '2024',
   '6.34',
   '2025',
   '2025',
   '2025',
   '2025',
   '2024',
   '2024'],
  'factual_difference': True},
 'contradiction': {'relationship': 'entailment',
  'confidence': 0.5525,
  'contradiction': False}}

1️⃣2️⃣ Chunking for Long Documents (Advanced)

Explanation

Long documents are split into chunks to improve embedding quality.

In [ ]:
def chunk_text(text, chunk_size=300):
    words = text.split()
    return [
        " ".join(words[i:i + chunk_size])
        for i in range(0, len(words), chunk_size)
    ]


Step 2: Chunk-wise Semantic Comparison

Explanation (for report)

Each chunk of document A is compared with all chunks of document B. The maximum similarity score is retained to represent the best semantic match.

In [ ]:
def chunked_semantic_similarity(text_a, text_b):
    chunks_a = chunk_text(text_a)
    chunks_b = chunk_text(text_b)

    embeddings_a = semantic_model.encode(chunks_a)
    embeddings_b = semantic_model.encode(chunks_b)

    similarity_matrix = cosine_similarity(embeddings_a, embeddings_b)

    # Best match for each chunk in A
    max_similarities = similarity_matrix.max(axis=1)

    return {
        "avg_similarity": round(float(np.mean(max_similarities)), 4),
        "min_similarity": round(float(np.min(max_similarities)), 4),
        "max_similarity": round(float(np.max(max_similarities)), 4)
    }


Step 3: Chunked Difference Decision

In [ ]:
def chunked_semantic_difference(text_a, text_b, threshold=0.8):
    stats = chunked_semantic_similarity(text_a, text_b)

    return {
        **stats,
        "semantic_difference": stats["avg_similarity"] < threshold
    }


5️⃣ Test Chunking on Resumes (THIS IS THE REAL TASK)

In [ ]:
chunked_semantic_difference(resume_old, resume_new)


{'avg_similarity': 0.995,
 'min_similarity': 0.995,
 'max_similarity': 0.995,
 'semantic_difference': False}

6️⃣ Why This Output Is Very Valuable for Your Report

You can now explain:

Average similarity → overall resume similarity

Minimum similarity → most changed section

Maximum similarity → unchanged section

This is much stronger than single-score comparison.

7️⃣ Combine Chunking with Other Differences (FINAL SYSTEM)

In [ ]:
def full_document_difference(text_a, text_b):
    return {
        "chunked_semantic": chunked_semantic_difference(text_a, text_b),
        "lexical": lexical_difference(text_a, text_b),
        "factual": factual_difference(text_a, text_b),
        "contradiction": contradiction_difference(text_a, text_b)
    }


In [ ]:
result = full_document_difference(text_a, text_b)
print(result)

{'chunked_semantic': {'avg_similarity': 0.7434, 'min_similarity': 0.7434, 'max_similarity': 0.7434, 'semantic_difference': True}, 'lexical': {'lexical_similarity': 0.7928, 'lexical_difference': True}, 'factual': {'numbers_text_a': ['92'], 'numbers_text_b': ['90'], 'factual_difference': True}, 'contradiction': {'relationship': 'contradiction', 'confidence': 0.9992, 'contradiction': True}}


{
  
  'chunked_semantic': {
        
        'avg_similarity': 0.7434,
        'min_similarity': 0.7434,
        'max_similarity': 0.7434,
        'semantic_difference': True
  },
   
   'lexical': {
         
         'lexical_similarity': 0.7928,
         'lexical_difference': True
  },
    
    'factual': {
          'numbers_text_a': ['92'],
           'numbers_text_b': ['90'],
            'factual_difference': True
  },
  'contradiction': {
           
           'relationship': 'contradiction',
            'confidence': 0.9992,
            'contradiction': True
  }

}
